# Sparse Score Fusion for Classifying Mate Pairs of Images


## Introduction

###  Challenge MDI343 2017-2018
#### Authors :  Umut Şimşekli & Stéphane Gentric


The topic of this challenge will be determining if two images belong to the same person or not. Conventionally, in order to solve this task, one typically builds an algorithm to provide a "score" for a given image pair. If the value of the score is high, it means that it is more probable that these images belong to the same person. Then, by using this score, one can determine if the images belong to the same person, by simply thresholding it.   

The goal of this challenge is to build a system for determining if two images belong to the same person or not by "fusing" multiple algorithms. In particular, for a given image pair, you will be provided the scores obtained from **14** different algorithms, each of which has a different computational complexity.   

Then the aim is to combine the scores of these algorithms (in a way that will be described in the sequel) in order to obtain a better classification accuracy. However, there will be a strict **computational budget**, such the running times of the algorithms that you combine **cannot exceed a certain time threshold**. For example, let $t_i$ denote the running time of algorithm $i$ in milliseconds ($i = 1,\dots,14$). Then, you will be given a threshold, $T$, such that the total computational time of the algorithms that you combine will not exceed $T$: 

$
\sum_{i\in C} t_i \leq T,
$

where $C \subset \{1,\dots,14\}$ is the set of algorithms that you choose to combine. The idea in such fusion is that "combining several fast algorithms might be better than using a single slow (possible more complex) algorithm". 

Before we describe how the fusion will be done, let us introduce the data:

**Training data:**

There will be $N= 2048853$ image pairs in the dataset. For a given image pair $n \in \{1,\dots,N\}$, we define $y_n = 1$ if this image pair belongs to the same person, or $y_n=0$ otherwise.

We then define a vector of scores for each image pair, $s_n \in \mathbb{R}_+^{14}$, such that $i$th component of $s_n$ will encode the score obtained by the $i$th algorithm, for the given image pair.  

**Test data:**

The test data contain $N_\text{test} = 170738$ image pairs. Similarly to the training data, each image pair contains a label and a vector of scores that are obtained from $14$ different algorithms. The test data will not be provided.


## Fusion Method 

In this challenge, you are expected to build a fusion system that is given as follows. Given a score vector $s \in \mathbb{R}_+^{14}$, we first define an extended vector $s'$, by appending a $1$ in the beginning of the original vector $s\in \mathbb{R}_+^{15}$: $s' = [1, s]$. Then we use the following fusion scheme in order to obtain the combined score $\hat{s}$: 

$
\hat{s} = s'^\top M s' 
$

where $M \in \mathbb{R}^{15 \times 15}$, is the "fusion matrix". This matrix will enable you to combine the scores of the different algorithms in a linear or a quadratic way. 


## The goal and the performance criterion

In this challenge, we will use an evaluation metric, which is commonly used in biometrics, namely the False Recognition Rate (FRR) at a fixed False Acceptance Rate (FAR). **The lower the better.**

The definitions of these quantities are as follows: (definitions from Webopedia)

**The false acceptance rate**, or **FAR**, is the measure of the likelihood that the biometric security system will incorrectly accept an access attempt by an unauthorized user. A system’s FAR typically is stated as the ratio of the number of false acceptances divided by the number of identification attempts.

**The false recognition rate**, or **FRR**, is the measure of the likelihood that the biometric security system will incorrectly reject an access attempt by an authorized user. A system’s FRR typically is stated as the ratio of the number of false recognitions divided by the number of identification attempts.

In this challenge, we will use the following evaluation scheme:

1) Given the scores, find a threshold that will give an FAR 0.01 %

2) Given the threshold, compute the FRR

The overall metric will be called **"the FRR at 0.01% FAR"**.


# Training Data

https://www.dropbox.com/s/6it6v6ifqkwuz98/train15_telecom.txt?dl=0


# Example Submission

In [17]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Running time of each algorithm (in milliseconds)
alg_times = np.zeros((14,1))
alg_times[0] = 163
alg_times[1] = 163
alg_times[2] = 190
alg_times[3] = 190
alg_times[4] = 206
alg_times[5] = 206
alg_times[6] = 120
alg_times[7] = 120
alg_times[8] = 83
alg_times[9] = 83
alg_times[10] = 83
alg_times[11] = 83
alg_times[12] = 170
alg_times[13] = 170

# Time constraint: The total duration of the algorithms cannot exceed 600 milliseconds
alg_time_thr = 600


# Compute the total computational time for the fusion algorithm
def compute_total_time(M):
    is_used = np.zeros((14,1))
    for i in range(15):
        for j in range(15):
            if(M[i,j] != 0):
                if(i>=1):
                    is_used[i-1] = 1
                if(j>=1):
                    is_used[j-1] = 1

    total_dur = np.dot(is_used.T,alg_times)
    return total_dur[0,0]

# Evaluation metric
def compute_eval(fused_score):
    look_at_FAR = 0.0001
    # calculating FAR and FRR
    sort = np.argsort(fused_score[:,1])

    #sort = np.concatenate([sort[-2:],sort[:-2]], axis=0)
    scores = fused_score[sort]
    totpos = sum(scores[:,0])
    totneg = scores.shape[0]-totpos
    fa = (np.cumsum(scores[:,0]-1)+totneg)/totneg
    fr = np.cumsum(scores[:,0])/totpos

    i=0
    while fa[i]>look_at_FAR:
        i+=1

    return scores[i][1], fa[i], fr[i]


In [2]:
# Load the data

train_fname = 'train15_telecom.txt'
train_data = np.loadtxt(train_fname, dtype=np.float) #The first column contains the labels, the rest of the columns contains the scores

# Extract the labels
y_trn = train_data[:,0].astype(int)

# Extract the score vectors
s_trn = train_data.copy()
# Put a 1 in front of all the scores (see the "Fusion method" section above)
s_trn[:,0] = 1;

In [29]:
#Prepare a fusion matrix
M = np.zeros((15,15))

#Example: the matrix will only average the first and the third algorithms:
M[0,1] = 0.5
M[0,3] = 0.5
 
#Example: Make the fusion for the first image pair:
cur_s = s_trn[0]
cur_s_hat = np.dot(cur_s.T,np.dot(M,cur_s)) 

#Check if the time constraint is satisfied:

tot_dur = compute_total_time(M)
print tot_dur

if(tot_dur <= alg_time_thr):
    print "The total running time of the fusion is acceptable!"
else:
    print "The total running time of the fusion is NOT acceptable!"


543.0
The total running time of the fusion is acceptable!


In [21]:
#Evaluation

#apply fusion on scores  
fuse = np.multiply(s_trn[:,None,:]*s_trn[:,:,None], M)
fuse = np.concatenate([np.reshape(y_trn, [-1,1]), np.reshape(np.sum(fuse, axis=(1,2)), [-1,1])], axis=1)
fuse[np.isnan(fuse)]=-float("inf")

#compute the FRR at FAR = 0.01%
thr, fa, fr = compute_eval(fuse)

print "Score at FAR="+str(look_at_FAR*100.0)+"%"
print "threshold :", thr, "far :",fa, "frr :", fr


Score at FAR=0.01%
threshold : 4879.43 far : 9.9610468009e-05 frr : 0.215112294649


In [30]:
#Submission

#Write the matrix M to the disk:
np.savetxt('M_pred.txt', M, fmt='%f')



Now, you can submit score_pred.txt to the challenge website.

Bonne chance !